# Import Library

In [1]:
## single frame analysis: whether there are large spatta?
# calculate the center position from image : sigle frame
import cv2
import numpy as np
import glob
import os
import csv
from matplotlib import pyplot as plt
import time
from numpy import dtype,uint8
import math

# Check start frame

## From side view

In [43]:
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\真横像\IR_1000W, B_500W, 100mm_s"
def check_center(index,threshold=60,thresholdArea=1000,rootDir=ROOT):
  """calculate center position

  Args:
      iindex (int): video index
      threshold(int): threshold for laser
      rootDir: video root path
      

  Returns:
      (list) : center position
  """
  #load data
  ## path to the movie
  path = os.path.join(rootDir,f"230823_{index}.avi")
  saveDir = os.path.join(rootDir,f"230823_{index}_center")
  if not os.path.exists(saveDir):
    os.makedirs(saveDir)
  #csv file
  filename=r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\230823_{index}_side.csv".format(index=index)
  #make csv file
  with open(filename,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Start","Finish"])
  #statistics save
  plotFile = os.path.join(saveDir,f"230823_{index}.jpg")
  #capture movie
  cap = cv2.VideoCapture(path)

  #list for saving center position
  X = []
  Y = []
  Areas = []
  count = 0
  startCounter=0
  finishCounter=0
  #calculate features
  #get image frames
  ret=True
  boolStart = False #to check when welding start
  boolFinish=False
  while ret :
      count += 1
      #print(f"count:{count},boolStart:{boolStart},boolFinish:{boolFinish}")
      #get frame
      ret, img = cap.read()
      if ret == False:
        break

      #if crop img, validate this code : [y_start:y_end,x_start:x_end,channel] , channel : grayScale->1,RGB->3
      #img = raw_img[yg-length:yg+length,xg-length:xg+length,:]}
      #img = img[yg-length:yg+length,xg-length:xg+length]
      # convert BGR to Gray
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      #crop image
      gray = gray[60:420,100:540]

      #if binarize img, validate this code. if brightness is greater than threshold, the pixel is set to 1, and othrewise 0
      #ret,thresh = cv2.threshold(gray,max/2,255,cv2.THRESH_BINARY)
      ret,thresh = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY)
      #find contours : 
      contours, hierarchy = cv2.findContours(np.array(thresh,dtype=uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


      Area = []
      #detect laser area
      if (bool(contours)==True):
        #面積(px*px)
        for j in contours:
          #calculate area of contours
          area = cv2.contourArea(j)
          Area.append(area)

        #Area is available
        if Area:
          ind = np.argmax(Area)
          cnt = contours[ind]
          Areas.append(Area[ind])

          if Area[ind]>thresholdArea and boolStart==False:
            startCounter=count
            boolStart=True
          if Area[ind]<thresholdArea and boolStart==True and boolFinish==False:
            finishCounter=count
            boolFinish=True
          #use image moment for calculating center position
          mu=cv2.moments(cnt)
          if mu["m00"] > 0:
            #gravity center
            gx=int(mu["m10"]/mu["m00"])
            gy=int(mu["m01"]/mu["m00"])
            X.append(gx)
            Y.append(gy)
            #save imgs to save directory
            if count % 100 == 50:
              cv2.circle(gray,(int(gx),int(gy)),3,color=(50,0,0),thickness=-1)
              cv2.imwrite(os.path.join(saveDir,f"{count:04d}.png"),gray) #cv2.imwrite(savePath,img) : os.path.join(RootDir,fileName) : concatenate file path. Here, RootDir+fileName
              #print("drewImg")

  #plot data
  fig,ax = plt.subplots(3,1,figsize=(6,6),tight_layout = True)
  ax[0].plot(X)
  ax[0].set_ylabel("X")
  ax[1].plot(Y)
  ax[1].set_ylabel("Y")
  ax[2].plot(Areas)
  ax[2].set_ylabel("Area")
  ax[2].set_xlabel("frame Index")
  ax[2].set_ylim(0,150)
  #fig.show()
  plt.savefig(plotFile)

  if boolFinish==False:
    finishCounter = -1
  if boolStart==False:
    startCounter = -1
  with open(filename,"a",newline='') as f:
      writer = csv.writer(f)
      writer.writerow([startCounter,finishCounter])

  #average mean position
  X=np.array(X)
  Y = np.array(Y)
  if X.size != 0:
    x_center_mean = X.mean()
    print(f"Center position : ({x_center_mean})")
  if Y.size != 0:
    y_center_mean = Y.mean()
    print(f"Center position : ({y_center_mean})")
  #this code is the same with the above code in numpy array
  #X = np.array(X) ## convert list to nd.array
  #Y = np.array(Y) ## convert list to nd.array
  #x_center_mean = X.mean()
  #y_center_mean = Y.mean()
  #if wanna calculate standard deviation
  #x_std = np.std(X)
  #y_std = np.std(Y)
  print(f"laser welding started from : {startCounter} and finished at {finishCounter}")

In [ ]:
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\side\IR_1000W, B_500W, 100mm_s"
for i in range(13,43):
    check_center(index=i,threshold=30,thresholdArea=20,rootDir=ROOT)
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\side\IR_1000W, B_500W, 200mm_s"
for i in range(51,81):
    check_center(index=i,threshold=30,thresholdArea=10,rootDir=ROOT)

## From Top view

In [45]:
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\top\IR_1000W, B_500W, 100mm_s"
def check_centerTop(index,threshold=60,thresholdArea=1000,rootDir=ROOT):
  """calculate center position

  Args:
      iindex (int): video index
      threshold(int): threshold for laser
      rootDir: video root path
      

  Returns:
      (list) : center position
  """
  #load data
  ## path to the movie
  path = os.path.join(rootDir,f"230823_{index}")
  path = os.path.join(path,"imgs")
  saveDir = os.path.join(rootDir,f"230823_{index}_center")
  if not os.path.exists(saveDir):
    os.makedirs(saveDir)
  #csv file
  filename=r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\230823_{index}_top.csv".format(index=index)
  #make csv file
  with open(filename,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Start","Finish"])
  #statistics save
  plotFile = os.path.join(saveDir,f"230823_{index}.jpg")
  #capture movie
  imageFiles = glob.glob(os.path.join(path, '*.png'))
  imageFiles.sort()

  #list for saving center position
  X = []
  Y = []
  Areas = []
  count = 0
  startCounter=0
  finishCounter=0
  #calculate features
  #get image frames
  ret=True
  boolStart = False #to check when welding start
  boolFinish=False
  for file in imageFiles:
      #print(file)
      count += 1
      #print(f"count:{count},boolStart:{boolStart},boolFinish:{boolFinish}")
      #get frame
      img = cv2.imread(file)
      # convert BGR to Gray
      if len(img.shape) != 3:
        gray = img
      else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      #crop image
      #gray = gray[60:420,100:540]

      #if binarize img, validate this code. if brightness is greater than threshold, the pixel is set to 1, and othrewise 0
      #ret,thresh = cv2.threshold(gray,max/2,255,cv2.THRESH_BINARY)
      ret,thresh = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY)
      #find contours : 
      contours, hierarchy = cv2.findContours(np.array(thresh,dtype=uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


      Area = []
      #detect laser area
      if (bool(contours)==True):
        #面積(px*px)
        for j in contours:
          #calculate area of contours
          area = cv2.contourArea(j)
          Area.append(area)

        #Area is available
        if Area:
          ind = np.argmax(Area)
          cnt = contours[ind]
          Areas.append(Area[ind])

          if Area[ind]>thresholdArea and boolStart==False:
            startCounter=count
            boolStart=True
          if Area[ind]<thresholdArea and boolStart==True and boolFinish==False:
            finishCounter=count
            boolFinish=True
          #use image moment for calculating center position
          mu=cv2.moments(cnt)
          if mu["m00"] > 0:
            #gravity center
            gx=int(mu["m10"]/mu["m00"])
            gy=int(mu["m01"]/mu["m00"])
            X.append(gx)
            Y.append(gy)
            #save imgs to save directory
            if count % 100 == 50:
              cv2.circle(gray,(int(gx),int(gy)),3,color=(50,0,0),thickness=-1)
              cv2.imwrite(os.path.join(saveDir,f"{count:04d}.png"),gray) #cv2.imwrite(savePath,img) : os.path.join(RootDir,fileName) : concatenate file path. Here, RootDir+fileName
              #print("drewImg")

  #plot data
  fig,ax = plt.subplots(3,1,figsize=(6,6),tight_layout = True)
  ax[0].plot(X)
  ax[0].set_ylabel("X")
  ax[1].plot(Y)
  ax[1].set_ylabel("Y")
  ax[2].plot(Areas)
  ax[2].set_ylabel("Area")
  ax[2].set_xlabel("frame Index")
  ax[2].set_ylim(0,1000)
  plt.show()
  plt.savefig(plotFile)

  with open(filename,"a",newline='') as f:
      writer = csv.writer(f)
      writer.writerow([startCounter,finishCounter])

  if boolFinish==False:
    finishCounter=-1
  #average mean position
  #average mean position
  X=np.array(X)
  Y = np.array(Y)
  if X.size != 0:
    x_center_mean = X.mean()
    print(f"Center position : ({x_center_mean})")
  if Y.size != 0:
    y_center_mean = Y.mean()
    print(f"Center position : ({y_center_mean})")
  #this code is the same with the above code in numpy array
  #X = np.array(X) ## convert list to nd.array
  #Y = np.array(Y) ## convert list to nd.array
  #x_center_mean = X.mean()
  #y_center_mean = Y.mean()
  #if wanna calculate standard deviation
  #x_std = np.std(X)
  #y_std = np.std(Y)
  print(f"laser welding started from : {startCounter} and finished at {finishCounter}")

In [ ]:
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\top\IR_1000W, B_500W, 100mm_s"
for i in range(13,43):
    check_centerTop(index=i,threshold=20,thresholdArea=100,rootDir=ROOT)
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\top\IR_1000W, B_500W, 200mm_s"
for i in range(51,81):
    check_centerTop(index=i,threshold=20,thresholdArea=100,rootDir=ROOT)

# remake movie so that each start frame match according to CSV file

In [ ]:
import pandas as pd
file = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308\230823_13_side.csv"
df = pd.read_csv(filepath_or_buffer=file)
data = df.values
print(data[0][0])

In [77]:
import pandas as pd
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308"

def synchronize2videos(index,topFPS=2000,sideFPS=8000,condition="IR_1000W, B_500W, 100mm_s",rootDir=ROOT):
    #csv file -> get start frame
    sideFile = os.path.join(rootDir,f"230823_{index}_side.csv")
    topFile = os.path.join(rootDir,f"230823_{index}_top.csv")
    side = pd.read_csv(filepath_or_buffer=sideFile)
    top = pd.read_csv(filepath_or_buffer=topFile)
    side = side.values
    top=top.values
    startSide = side[0][0]
    startTop = top[0][0]
    #video
    #top-view images
    pathTop = os.path.join(rootDir,"top\\{condition}\\230823_{index}\\imgs".format(condition=condition,index=index))
    imageFiles = glob.glob(os.path.join(pathTop, '*.png'))
    imageFiles.sort() 
    saveDirTop = os.path.join(rootDir,"synchronization\\top\\{condition}\\{index}".format(condition=condition,index=index))
    if not os.path.exists(saveDirTop):
        os.makedirs(saveDirTop)
    #side view
    pathSide = os.path.join(rootDir,"side\\{condition}\\230823_{index}.avi".format(condition=condition,index=index))
    saveDirSide = os.path.join(rootDir,"synchronization\\side\\{condition}\\{index}".format(condition=condition,index=index))
    if not os.path.exists(saveDirSide):
        os.makedirs(saveDirSide)
    #top view 
    #save images
    count=0
    print("topView")
    for file in imageFiles:
      #print(file)
      idx = int((file.split("\\")[-1]).split(".")[0])
      #print(f"count:{count},boolStart:{boolStart},boolFinish:{boolFinish}")
      #get frame
      img = cv2.imread(file)
      # convert BGR to Gray
      if len(img.shape) != 3:
        gray = img
      else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      if (idx+1)>=startTop:
         cv2.imwrite(os.path.join(saveDirTop,f"{idx:05d}.png"),gray)
    #side view
    #get image frames
    
    ret=True
    boolStart = False #to check when welding start
    cap = cv2.VideoCapture(pathSide)
    print("Side view")
    while ret :
        count += 1
        #print(count)
        #print(f"count:{count},boolStart:{boolStart},boolFinish:{boolFinish}")
        #get frame
        ret, img = cap.read()
        if ret == False:
           break
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #crop image
        gray = gray[50:430,50:580]
        if count>=startSide:
            cv2.imwrite(os.path.join(saveDirSide,f"{count:05d}.png"),gray)
            '''for keeping frame consistent with top view videos
            if not boolStart: #first time
              cv2.imwrite(os.path.join(saveDirSide,f"{count:05d}.png"),gray)
              boolStart=True
            else:#after 2nd time
              if ((count-startSide)%int(sideFPS//topFPS)==0):
                 cv2.imwrite(os.path.join(saveDirSide,f"{count:05d}.png"),gray)  
            '''
    

In [78]:
ROOT = r"C:\Users\Furukawa\Documents\TechAssistant\Video\202308"
for i in range(13,43):
    synchronize2videos(index=i,topFPS=2000,sideFPS=8000,condition="IR_1000W, B_500W, 100mm_s",rootDir=ROOT)
for i in range(51,81):
    synchronize2videos(index=i,topFPS=2000,sideFPS=8000,condition="IR_1000W, B_500W, 200mm_s",rootDir=ROOT)

topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Side view
topView
Si

# Make movie from images

In [85]:
import glob

import cv2
import os
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

#src = r'C:\Users\kawaw\cpp\yolov8PoseEstimation\yolov8PoseEstimation\video\poseEstimation'
SRC = r'C:\Users\Furukawa\Documents\TechAssistant\Video\202308\synchronization\side\IR_1000W, B_500W, 100mm_s'
def makeVideo(index,direction="side",condition="IR_1000W, B_500W, 100mm_s"):
    src=r'C:\Users\Furukawa\Documents\TechAssistant\Video\202308\synchronization\{direction}\{condition}\{index}'.format(direction=direction,condition=condition,index=index)
    img_array = []
    for filename in sorted(glob.glob(f"{src}/*.png"), key=natural_keys):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)
    if direction=="side":
        dir=r'C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\{condition}\side'.format(condition=condition)
        if not os.path.exists(dir):
            os.makedirs(dir)
        name = r'C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\{condition}\side\{index}.avi'.format(condition=condition,index=index)
        out = cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'XVID'), 30.0, (530,380)) #(x,y)
    elif direction=="top":
        dir=r'C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\{condition}\video'.format(condition=condition)
        if not os.path.exists(dir):
            os.makedirs(dir)
        name = r'C:\Users\Furukawa\Documents\TechAssistant\Video\20230823_synchronized\{condition}\video\{index}.avi'.format(condition=condition,index=index)
        out = cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'XVID'), 30.0, (160,160))

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [88]:
directions=["side","top"]
#for i in range(13,43):
    #for dir in directions:
       # makeVideo(index=i,direction=dir,condition="IR_1000W, B_500W, 100mm_s")
for j in range(51,81):
    for dir in directions:
        makeVideo(index=j,direction=dir,condition="IR_1000W, B_500W, 200mm_s")